In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from sklearn.decomposition import PCA
import math
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap
from sklearn.decomposition import KernelPCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn import mixture
from sklearn.manifold import TSNE, MDS
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import LocallyLinearEmbedding, trustworthiness
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csgraph, csc_matrix
import scipy.sparse.linalg as LA
import collections
from sklearn.neighbors import kneighbors_graph
import scipy

def visualize_4d(frame, hot=True):
  fig = plt.figure(figsize=(7, 7))
  ax = fig.add_subplot(111, projection='3d')
  x = np.array(frame.iloc[:,0])
  if len(frame.columns) > 1:
    y = np.array(frame.iloc[:,1])
  if len(frame.columns) > 2:
    z = np.array(frame.iloc[:,2])
  if len(frame.columns) == 1:
    img = ax.scatter(x, x, s=2)
  if len(frame.columns) == 2:
    img = ax.scatter(x, y, s=2)
  elif len(frame.columns) == 3:
    img = ax.scatter(x, y, z, s=2)
  else:
    c = np.array(frame.iloc[:,3])
    if hot:
      img = ax.scatter(x, y, z, c=c, cmap=plt.hot(), s=2)
    else:
      img = ax.scatter(x, y, z, c=c, cmap='viridis', s=2)
  plt.show()

In [67]:
# Input
file = open('./R31.txt','r')
d, n, m, k, p = list(file.readline().split())
d, n, p, k = map(int, [d, n, p, k])
k_list = list(map(int, file.readline().split()))
ar = []
for i in range(n):
  ar.append(list(map(float, file.readline().split())))
df = pd.DataFrame(ar)
df.describe()

,0,1,2
count,29678.000000,29678.000000,29678.000000
mean,419.808233,-69.027823,31.336202
std,255.848084,62.353086,67.380433
min,-29.291716,-205.376219,-101.377696
25%,195.227485,-112.720350,-25.854263
50%,443.645602,-65.691209,21.230403
75%,644.640312,-23.550395,87.041762
max,792.444881,69.779176,156.502445


In [68]:
def find_best_eps(min_sample):
  print(f"samples: {min_sample}")
  tmp = df[[i for i in range(d)]].copy()
  l, r = 0, 200
  while (r - l) > 0.01:
    mid = (r + l) / 2
    print(mid)
    model_name = 'dbscan'
    model = DBSCAN(eps=mid, min_samples=min_sample)
    model.fit(tmp[[i for i in range(d)]])
    tmp['subset'] = model.labels_
    print(f"outlier count: {len(tmp[tmp['subset'] == -1])}")
    print(f"manifold count: {tmp['subset'].max() + 1}")
    if len(tmp[tmp['subset'] == -1]) > p:
      l = mid
    else:
      r = mid
  return r

# Finding subsets
samples = 4
best_eps = find_best_eps(samples)
#best_eps = 82.03125
model = DBSCAN(eps=best_eps, min_samples=2)
model.fit(df[[i for i in range(d)]])
df['manifold'] = model.labels_

samples: 4
100.0
outlier count: 0
manifold count: 3
50.0
outlier count: 0
manifold count: 8
25.0
outlier count: 0
manifold count: 10
12.5
outlier count: 0
manifold count: 11
6.25
outlier count: 1
manifold count: 15
9.375
outlier count: 0
manifold count: 11
7.8125
outlier count: 0
manifold count: 14
7.03125
outlier count: 0
manifold count: 15
6.640625
outlier count: 1
manifold count: 15
6.8359375
outlier count: 0
manifold count: 15
6.73828125
outlier count: 0
manifold count: 15
6.689453125
outlier count: 0
manifold count: 15
6.6650390625
outlier count: 1
manifold count: 15
6.67724609375
outlier count: 0
manifold count: 15
6.671142578125
outlier count: 0
manifold count: 15


In [69]:
%matplotlib inline
def get_local_intrinsic_dimension(data, n_neighbors=5):
  X = np.array(data)
  M = np.matrix(X)
  L = n_neighbors
  if not isinstance(L, (int, np.integer))  or L<0:
    return 'n_neighbors must be a positive integer!'
  neigh = NearestNeighbors(n_neighbors=L)
  nbrs = neigh.fit(X)
  distances, indices = nbrs.kneighbors()
  mean = [np.mean([M[indices[i][j]] for j in range(L)],axis=0) for i in range(len(X))]
  # Calculating the local covariance matrix for each point
  C = [1/L * sum([np.dot( (M[indices[i][j],:]-mean[i]).transpose() , (M[indices[i][j],:]-mean[i]) ) for j in range(L)]) for i in range(len(X))]

  # Intrinsic Dimension Estimation
  THRESHOLD = 0.05
  intrinsic_dimension = [0] * len(X)

  E = [sorted(scipy.linalg.eigh(C[i])[0], reverse=True) for i in range(len(X))]
  for i in range(len(X)):
    eigen_list = E[i]
    d = len(eigen_list)
    first_eigenvalue= eigen_list[0]
    for j in range(1,d):
      if eigen_list[j]/first_eigenvalue < THRESHOLD:
        intrinsic_dimension[i]=j
        break
    if intrinsic_dimension[i]==0:
      intrinsic_dimension[i]=d
  return np.array(intrinsic_dimension)

def get_dim(X, threshold=0.95, max_d=15, print_scores=False):
  for td in range(1, max_d):
    res = pd.DataFrame(LocallyLinearEmbedding(n_components=td, n_neighbors=90, max_iter=1000).fit_transform(X),
                       columns=[i for i in range(td)])
    score = trustworthiness(X, res, n_neighbors=100)
    if print_scores:
      print(f'dim score {td}: {score}')
    if score >= threshold:
      return td
  return max_d

# Assuming data is consisted of sub-manifolds with DIFFERENT dimensions OR is just consisted of one manifold
def is_manifold(data, n_neighbors=5, error=0.05, print_accuracy=False):
  ID = get_local_intrinsic_dimension(data, n_neighbors=n_neighbors)
  total = len(ID)
  counter = collections.Counter(ID)
  print(counter)
  dimension = counter.most_common(1)[0][0]
  accuracy = counter[dimension]/total
  if accuracy > 1 - error:
    if print_accuracy:
     print(f'Accuracy: {accuracy}. Data resides on a {dimension}-dimensional manifold!')
    return True, accuracy, dimension
  if print_accuracy:
    print(f'Accuracy: {accuracy} -> Inconclusive!')
  return False, accuracy, dimension

multi = None # subsets containing multiple manifolds
if multi is None:
  multi = []
  for i in range(int(df['manifold'].max()) + 1):
    print(i)
    X = df[df['manifold'] == i][[i for i in range(d)]]
    dim = get_dim(X)  # could be replaced with d when d is small
    b0, a0, d0 = is_manifold(X, n_neighbors=dim * 2)
    b1, a1, d1 = is_manifold(X, n_neighbors=dim * 3)
    if not b0 and not b1:
      multi.append(i)
      td = 3
      res = pd.DataFrame(LocallyLinearEmbedding(n_components=td, n_neighbors=20, max_iter=10000).fit_transform(X), columns=[i for i in range(td)])
      visualize_4d(res)
      visualize_4d(X)

print(multi)
# TODO doesn't work very well maybe add dimension dfs?

0
Counter({1: 1804})
Counter({1: 1804})
1
Counter({1: 1804})
Counter({1: 1804})
2
Counter({1: 1871})
Counter({1: 1871})
3
Counter({1: 1893})
Counter({1: 1893})
4
Counter({1: 939})
Counter({1: 939})
5
Counter({1: 1893})
Counter({1: 1893})
6
Counter({1: 945})
Counter({1: 945})
7
Counter({1: 1828})
Counter({1: 1828})
8
Counter({1: 1844})
Counter({1: 1844})
9
Counter({1: 1828})
Counter({1: 1828})
10
Counter({2: 1539, 1: 306})
Counter({2: 1799, 1: 46})
11
Counter({1: 1889})
Counter({1: 1889})
12
Counter({1: 1889})
Counter({1: 1889})
13
Counter({1: 3762})
Counter({1: 3762})
14
Counter({1: 3644})
Counter({1: 3644})
[]


In [70]:
%matplotlib inline
class ComponentScan:

  def __init__(self, n_neighbor=10, step=20, max_rep=10):
    self.neighbors = None
    self.points = None
    self.n_neighbor = n_neighbor
    self.step = step
    self.components_ = None
    self.max_rep = max_rep

  def fit(self, X):
    self.neighbors = []
    for i in range(self.max_rep):
      nbrs = NearestNeighbors(n_neighbors=self.n_neighbor + i * self.step, algorithm='brute').fit(X)
      self.neighbors.append(nbrs.kneighbors(X)[1])
    self.points = X.to_numpy()
    self.components_ = np.full((len(X)), -1)
    return self

  def predict(self):
    cnt = 0
    ind = 0
    c = -1
    rep = 0
    while True:
      while ind < len(self.points):
        if self.components_[ind] == -1:
          break
        ind += 1
      if ind == len(self.points):
        break
      stack = [ind]
      combo = []
      cnt = 0
      c += 1
      while len(stack) > 0:
        cnt += 1
        if cnt % 1000 == 0:
          print(f'{cnt} - {len(stack)}')
        v = stack.pop()
        self.components_[v] = c
        for u in self.neighbors[rep][v]:
          if self.components_[u] == -1:
            stack.append(u)
            combo.append(u)
            self.components_[u] = c
      if len(combo) < 100:
        rep += 1
        if rep == self.max_rep:
          rep = 0
        for c in combo:
          self.components_[c] = -1
      else:
        rep = 0
    return c + 1

sp = df.copy()
for subset in multi:
  print(f'processing subset {subset}')
  tmp = df[df['manifold'] == subset][[i for i in range(d)]]
  td = 3
  print(f'dim: {td}')
  res = pd.DataFrame(LocallyLinearEmbedding(
    n_components=td,
    n_neighbors=20,
    max_iter=1000
  ).fit_transform(
    tmp
  ), columns=[i for i in range(td)])
  # visualize_4d(res)
  print(f'LLE done')
  compo = ComponentScan(n_neighbor=40, step=100).fit(res)
  k = compo.predict()
  print(f'manifold count: {k}')
  if k > 10:
    continue
  sp.loc[sp['manifold'] == subset, 'sub1'] = compo.components_
  print(sp.loc[sp['manifold'] == subset, 'sub1'].value_counts())
  visualize_4d(
    pd.DataFrame(
      PCA(n_components=3).fit_transform(tmp),
      columns=[j for j in range(3)]
    ),
    hot=False
  )
if len(multi) > 0:
  sp['manifold'] = sp.manifold.astype(str) + '-' + sp.sub1.astype(str)
  sp = sp.drop(['sub1'], axis=1)
  sp['manifold'] = LabelEncoder().fit_transform(sp['manifold'])
  sp['manifold'] -= 1
  sp['manifold'].describe()

In [ ]:
# Custom cluster for manifold
final_k_list = []
cluster_type = []
for i in range(int(sp.manifold.max()) + 1):
  print(i)
  X = sp[sp['manifold'] == i][[j for j in range(d)]]
  td = max(get_dim(X, max_d=7), 3)
  res = pd.DataFrame(LocallyLinearEmbedding(
    n_components=td,
    n_neighbors=50,
    max_iter=1000
  ).fit_transform(
    X
  ), columns=[i for i in range(td)])
  k = ComponentScan(n_neighbor=40, step=30).fit(res).predict()
  if k < 10:
    cluster_type.append('custom')
  else:
    cluster_type.append('HS')
  final_k_list.append(k if k < 10 else 1)
xsum = 0
for c in final_k_list:
  xsum += c
print(xsum)

0
1000 - 805
1
1000 - 805
2
1000 - 40
3
1000 - 433
4
5
1000 - 489
6
7
1000 - 829
8
9
1000 - 829
10
1000 - 846
11
1000 - 104
12
1000 - 890
13


In [ ]:
%matplotlib inline
def most_common_dimension(data, n_neighbors=5):
  ID = get_local_intrinsic_dimension(data, n_neighbors=n_neighbors)
  total = len(ID)
  # n_manifolds = max(ID)
  counter = collections.Counter(ID)
  dimension = counter.most_common(1)[0][0]
  return dimension,counter[dimension]/total

def estimate_intrinsic_dimension(data,n_neighbors=5, method='most_common'):
  res = 0
  if method=='most_common':
    return most_common_dimension(data,n_neighbors)[0]
  if method=='average':
    return np.around(np.mean(get_local_intrinsic_dimension(data),axis=0),0).astype(int)

plt.close('all')
D = []
for i in range(int(df.manifold.max()) + 1):
  print(i)
  X = df[df['manifold'] == i][[j for j in range(d)]]
  dim1 = get_dim(X, max_d=6, print_scores=True)
  if dim1 < 6:
    D.append(dim1)
  else:
    D.append(estimate_intrinsic_dimension(X, n_neighbors=2 * d))
  print(D[-1])
print(D)

In [ ]:
# Joining the cluster labeling
def get_cluster(X, num_cluster, cluster_type):
  if cluster_type == 'GMM':
    model = mixture.GaussianMixture(n_components=num_cluster, covariance_type='full', n_init=100)
    model.fit(X)
    labels = model.predict(X)
  elif cluster_type == 'k-means':
    model=KMeans(n_clusters=num_cluster, n_init=10, max_iter=10000)
    model.fit(X)
    labels = model.predict(X)
  elif cluster_type == 'HS':
    model = AgglomerativeClustering(n_clusters=num_cluster, linkage='single')
    model.fit(X)
    labels = model.labels_
  elif cluster_type == 'H':
    model = AgglomerativeClustering(n_clusters=num_cluster)
    model.fit(X)
    labels = model.labels_
  elif cluster_type == 'spectral':
    model = SpectralClustering(assign_labels='discretize', n_clusters=num_cluster, random_state=77, n_init=1)
    model.fit(X)
    labels = model.labels_
  elif cluster_type == 'custom-scan':
    td = get_dim(X)
    res = pd.DataFrame(LocallyLinearEmbedding(
      n_components=td,
      n_neighbors=d // 2,
      max_iter=100000
    ).fit_transform(
      X
    ), columns=[i for i in range(td)])
    # visualize_4d(res)
    compo = ComponentScan(n_neighbor=50, step=50).fit(res)
    compo.predict()
    return compo.components_
  elif cluster_type == 'PCAH':
    tmp = pd.DataFrame(PCA(
      n_components=3,
    ).fit_transform(
      X
    ), columns=[i for i in range(3)])
    return get_cluster(tmp, num_cluster, 'GMM')
  elif cluster_type == 'custom':
    td = 2
    p = pd.DataFrame(LocallyLinearEmbedding(n_components=td, n_neighbors=d, eigen_solver='dense').fit_transform(StandardScaler().fit_transform(X)), columns=[j for j in range(td)])
    return get_cluster(p, cluster_type='GMM', num_cluster=num_cluster)
  elif cluster_type == 'custom3d':
    td = 3
    p = pd.DataFrame(LocallyLinearEmbedding(n_components=td, n_neighbors=d).fit_transform(StandardScaler().fit_transform(X)), columns=[j for j in range(td)])
    return get_cluster(p, cluster_type='GMM', num_cluster=num_cluster)
  elif cluster_type == 'custom50':
    td = 2
    p = pd.DataFrame(LocallyLinearEmbedding(n_components=td, n_neighbors=2 * d).fit_transform(StandardScaler().fit_transform(X)), columns=[j for j in range(td)])
    return get_cluster(p, cluster_type='GMM', num_cluster=num_cluster)
  return labels

def join_clusters():
  df['cluster'] = [0]*n
  data_index = [0] * (int(df.manifold.max()) + 1)
  for i in range(n):
    index_manifold = int(df.at[i,'manifold'])
    if index_manifold!=-1:
      df.at[i,'cluster'] = list_labels[index_manifold][data_index[index_manifold]]
      data_index[index_manifold]+=1

list_labels = []
for i in range(int(df.manifold.max()) + 1):
  print(i)
  if final_k_list[i] == 1:
    list_labels.append([0] * len(df[df['manifold'] == i]))
    continue
  labels = get_cluster(df[df['manifold'] == i][[i for i in range(d)]], num_cluster=final_k_list[i], cluster_type=cluster_type[i])
  list_labels.append(labels)
print('------')
print(int(df.manifold.max()) + 1)
print('-------')
join_clusters()

In [ ]:
import miniball

def get_pca(x):
    pca = PCA(n_components = d)
    pca.fit_transform(x)
    return pca.explained_variance_ratio_, pca.components_

THRESHOLD = 0.99
def find_normal_vectors(eigens,vectors):
  res = sum(eigens)
  current = 0
  normal_vectors = []
  for i in range(0,len(eigens)):
    current += eigens[i]
    if current/res > THRESHOLD:
      for j in range(i+1,len(eigens)):
        normal_vectors.append(vectors[j])
      break
  return normal_vectors
# Outputting
def get_affine_space(points):
  X = points
  s, v = get_pca(X)
  #Finding Normal Vectors using PCA
  A = find_normal_vectors(s, v)
  #Finding b-s using PCA
  b = []
  mean = np.mean(points, axis = 0)
  for j in range(len(A)):
    b.append(np.dot(A[j],mean))
  return A, b
def get_optimal_miniball(points):
  points = np.asarray(points)
  minlist = [min(points, key=lambda p: p[j])[j] for j in range(0,d)]
  points = [[p[j]-minlist[j] for j in range(0,d)] for p in points]
  mb = miniball.Miniball(points)
  c = mb.center()
  for j in range(0,d):
    c[j]+=minlist[j]
  r = math.sqrt(mb.squared_radius())
  if not mb.is_valid():
    print('Possibly invalid!')
  print('Relative error', mb.relative_error())
  return c,r
def spherical_measure(data):
  X = np.array(data)
  n = len(X)
  d = len(X[0])
  c,r = get_optimal_miniball(X)
  c = np.array(c)
  r = np.array(r)
  SSE = np.array([(np.linalg.norm(X[i]-c)-r)**2 for i in range(n)]).sum()
  total = n * (r**2)
  print(f'error divided by total: {SSE / total}')
  print(f'error: {SSE / n}')
  return SSE/total
def get_manifold_type(data, mcl_d, acceptable_error=1e-3):
  X = data.to_numpy()
  pca_d = d-len(find_normal_vectors(get_pca(X)[0], get_pca(X)[1]))
  if spherical_measure(data) < acceptable_error:
    print(f'{pca_d} - {mcl_d}')
    return 'Sphere'
  if pca_d > mcl_d:
    return 'Complex'
  return 'Affine'

vectors = [[[] for j in range(0,final_k_list[i])] for i in range(int(df.manifold.max()) + 1)]
#df.loc[df['column_name'] == some_value]
outlier = []
for i in range(n):
  if int(df.at[i, 'manifold']) == -1:
    outlier.append(i + 1)
  else:
    vectors[int(df.at[i, 'manifold'])][int(df.at[i, 'cluster'])].append(i + 1)

with open('./output.txt', 'w') as f:
  print(f'{n} {int(df.manifold.max()) + 1}', file=f)
  for i in range(int(df.manifold.max()) + 1):
    print(i)
    X = df[df['manifold'] == i][[j for j in range(d)]]
    if D[i] == 6:
      D[i] = get_dim(X, max_d=15)
    manifold_type = get_manifold_type(X, D[i])
    if manifold_type == 'Complex':
      dimension = D[i]
    else:
      dimension = d - len(find_normal_vectors(get_pca(X)[0], get_pca(X)[1]))
    print(f'{dimension} ' + str(final_k_list[i]) + f' {manifold_type}', file=f)
    if manifold_type != 'Complex':
      A, B = get_affine_space(X)
      for j in range(len(A)):
        print(' '.join(list(map(str,A[j]))), file=f)
      print(' '.join(list(map(str,B))), file=f)
    if manifold_type == 'Sphere':
      c, r = get_optimal_miniball(X)
      print(' '.join(list(map(str, c))) + ' ' + str(r), file=f)
    for j in range(0,final_k_list[i]):
      print(str(len(vectors[i][j])) + ' ' + ' '.join(list(map(str, vectors[i][j]))), file=f)
  print(str(len(outlier)) + ' ' + ' '.join(list(map(str, outlier))), file=f)